In [1]:
%matplotlib inline

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Layer
from keras.layers import Conv2D, Conv2DTranspose, Conv1D, LSTM
from keras.models import Model
from keras import backend as K
from keras import metrics
from scipy.stats import norm

# Image Parameters

In [8]:
# input image dimensions
img_rows, img_cols, img_chns = 28, 28, 1

# number of convolutional filters to use
filters = 64

# convolution kernel size
num_conv = 3

batch_size = 100
if K.image_data_format() == 'channels_first':
    original_img_size = (img_chns, img_rows, img_cols)
else:
    original_img_size = (img_rows, img_cols, img_chns)

# Import MNIST Images

In [9]:
from keras.datasets import mnist

In [10]:
(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((x_train.shape[0],) + original_img_size)
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape((x_test.shape[0],) + original_img_size)

print('x_train.shape:', x_train.shape)

('x_train.shape:', (60000, 28, 28, 1))


### Convert to shape of multivariate time series

In [11]:
x_train = np.asarray([x_train[i,:,:,:].reshape(28*28) for i in range(x_train.shape[0])])
x_test = np.asarray([x_test[i,:,:,:].reshape(28*28) for i in range(x_test.shape[0])])

# Simple LSTM
Without "return sequence" not "stateful" parameter

In [44]:
x = Input(shape=(10,28*28))
lstm_1 = LSTM(32)(x)
model = Model(x,lstm_1)
model.compile(optimizer="adam", loss="mse")

In [47]:
model.predict(x_train[0:30].reshape(3,10,784)).shape

(3, 32)

# Simple stateful LSTM
Without "return sequence"
Stateful LSTM share their internal states between examples

In [55]:
x = Input(shape=(10,28*28),batch_shape=(3,10,28*28))
lstm_1 = LSTM(32,
              stateful=True)(x)
model = Model(x,lstm_1)
model.compile(optimizer="adam", loss="mse")

In [56]:
model.predict(x_train[0:30].reshape(3,10,784)).shape

(3, 32)

# Simple return sequence LSTM 
Without "stateful"

In [60]:
x = Input(shape=(10,28*28))
lstm_2 = LSTM(32, return_sequences=True)(x)
model = Model(x,lstm_2)
model.compile(optimizer="adam", loss="mse")

In [62]:
model.predict(x_train[0:10].reshape(1,10,784)).shape

(1, 10, 32)

# Stacked LSTM

In [63]:
x = Input(shape=(10,28*28))
lstm_2 = LSTM(32, return_sequences=True)(x)
lstm_3 = LSTM(15, return_sequences=True)(lstm_2)
lstm_4 = LSTM(5)(lstm_3)
model = Model(x,lstm_4)
model.compile(optimizer="adam", loss="mse")

In [64]:
model.predict(x_train[0:10].reshape(1,10,784)).shape

(1, 5)

# Auto Encoder

In [ ]:
inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)